# Multimodal detection with MDETR in TorchMultimodal

[MDETR](https://arxiv.org/pdf/2104.12763.pdf) (Kamath et al.) is a multimodal detector that detects objects in an image conditioned on text queries. It is an extension of the [DETR](https://arxiv.org/pdf/2005.12872.pdf) model, which treats object detection as a set prediction problem, performing a bipartite matching between the ground truth and predicted bounding boxes. MDETR extends this approach to vision & language tasks with the addition of a contrastive alignment loss and a soft token prediction loss.

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url="https://production-media.paperswithcode.com/methods/Screen_Shot_2021-08-11_at_10.03.50_AM.png")

MDETR shows strong performance on a number of downstream tasks, including visual question answering (VQA), phrase grounding, and referring expression comprehension and segmentation. In this tutorial, we first give an introduction to the components of MDETR in TorchMultimodal, then provide a walkthrough of how to apply MDETR on both phrase grounding and VQA tasks.

## MDETR in TorchMultimodal

TorchMultimodal provides access to MDETR components at multiple different levels of granularity. This allows users to easily plug in their own modules for experimentation without needing to change the underlying implementation.

In [ ]:
# Simplest entry point: model builder function.
# Provides prebuilt model from the paper with ResNet101 backbone.
from torchmultimodal.models.mdetr.model import mdetr_resnet101
mdetr_rn101 = mdetr_resnet101()

# The builder function contains sensible default values but can also be easily modified.
# E.g. we can modify the number of object queries and change the transformer FFN dimension
modified_mdetr_rn101 = mdetr_resnet101(num_queries=100, transformer_dim_feedforward=1024)

In [ ]:
# mdetr_resnet101 is an instantiation of the MDETR model class.
# This class is composed of modular components which can be swapped out for easy experimentation.
from torchmultimodal.models.mdetr.model import MDETR

# Other imports we will use
import math
from torch import nn
from torchmultimodal.modules.layers.mlp import MLP
from torchmultimodal.models.mdetr.model import FeatureResizer
from torchmultimodal.models.mdetr.image_encoder import mdetr_resnet101_backbone, MaskedIntermediateLayer, PositionEmbedding2D
from torchmultimodal.models.mdetr.text_encoder import mdetr_roberta_text_encoder
from torchmultimodal.models.mdetr.transformer import mdetr_transformer

# Set some dimensions as constant
image_embedding_dim = 2048
text_embedding_dim = 768
transformer_dim = 256
num_queries = 100
num_classes = 255

# The core components of the MDETR model class are:
#   - image backbone 
#   - text encoder 
#   - multimodal transformer

# Encoder and transformer components also have builder functions.
image_backbone = mdetr_resnet101_backbone()
text_encoder = mdetr_roberta_text_encoder()
transformer = mdetr_transformer()


# Other components include
#   - positional embeddings: 2D embeddings added to image backbone outputs
#   - text/image projections (between unimodal encoders and multimodal transformer)
#   - query embeddings: learned object query embeddings, used in multimodal transformer decoder
#   - bounding box embeddings: map multimodal transformer outputs to bounding box coordinates
#   - class embeddings: map multimodal transformer outputs to class logits 

pos_embed = PositionEmbedding2D(num_pos_feats=128, scale=2 * math.pi)
text_projection = FeatureResizer(text_embedding_dim, transformer_dim)
image_projection = nn.Conv2d(image_embedding_dim, transformer_dim, kernel_size=1)
query_embed = nn.Embedding(num_queries, transformer_dim)
bbox_embed = MLP(transformer_dim, 4, [transformer_dim] * 2, dropout=0.0)
class_embed = nn.Linear(transformer_dim, num_classes + 1)

# This will give us the original ResNet101 MDETR model
mdetr = MDETR(
    image_backbone = image_backbone,
    text_encoder = text_encoder, 
    transformer = transformer,
    pos_embed = pos_embed, 
    text_projection = text_projection,
    image_projection = image_projection, 
    query_embed = query_embed,
    bbox_embed = bbox_embed,
    class_embed = class_embed,
)

# Swap out text encoder and class embedding for other components
from transformers import DistilBertModel
distilbert_text_encoder = DistilBertModel.from_pretrained("distilbert-base-uncased")
mlp_class_embed = MLP(transformer_dim, num_classes + 1, [transformer_dim] * 2, activation=nn.GELU, dropout=0.1)
modified_mdetr = MDETR(
    image_backbone = image_backbone,
    text_encoder = distilbert_text_encoder, 
    transformer = transformer,
    pos_embed = pos_embed, 
    text_projection = text_projection,
    image_projection = image_projection, 
    query_embed = query_embed,
    bbox_embed = bbox_embed,
    class_embed = class_embed,
)


## Phrase grounding

This section demonstrates how to use MDETR for [phrase grounding](https://paperswithcode.com/task/phrase-grounding/codeless). In phrase grounding, the goal is to associate noun phrases in the caption of an `(image, text)` pair to regions in the image. This is in line with MDETR's soft token alignment loss, which means we can perform inference on this task directly from the pretrained checkpoint. Here we will use a few samples from the [Flickr30k](https://paperswithcode.com/dataset/flickr30k#:~:text=Introduced%20by%20Young%20et%20al,sentences%20provided%20by%20human%20annotators.) validation set.

In [ ]:
# TorchMultimodal also provides entry points to apply MDETR on specific tasks.
# These wrap the MDETR class from the preceding section with task-specific losses and/or heads.
import torch
from torchmultimodal.models.mdetr.model import mdetr_for_phrase_grounding

mdetr_pg = mdetr_for_phrase_grounding()
checkpoint = torch.hub.load_state_dict_from_url("https://pytorch.s3.amazonaws.com/models/multimodal/mdetr/pretrained_resnet101_checkpoint.pth")
mdetr_pg.load_state_dict(checkpoint["model_ema"])
mdetr_pg.eval()
print("done")

In [ ]:
# Download a few sample images and corresponding annotations
import os
asset_dir = "assets/phrase_grounding"
img_dir = os.path.join(asset_dir, "images")
os.makedirs(img_dir, exist_ok=True)

!wget -nc "https://pytorch.s3.amazonaws.com/torchmultimodal/examples/mdetr/assets/phrase_grounding/images/4287766241.jpg" -P $img_dir
!wget -nc "https://pytorch.s3.amazonaws.com/torchmultimodal/examples/mdetr/assets/phrase_grounding/images/4591476340.jpg" -P $img_dir
!wget -nc "https://pytorch.s3.amazonaws.com/torchmultimodal/examples/mdetr/assets/phrase_grounding/flickr30k_sample_annotations.json" -P $asset_dir

In [ ]:
from examples.mdetr.data.dataset import ModulatedDetection
import torchvision.transforms as T
from transformers import RobertaTokenizerFast

# Construct tokenizer, image transform, and dataset on sample data 
# We can also pass transforms directly to the dataset class via MDETRTransform. 
# We opt not to so that we can inspect each sample first.
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
img_transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
dataset = ModulatedDetection(
        img_dir,
        os.path.join(asset_dir, "flickr30k_sample_annotations.json"),
        transforms=None,
        return_tokens=True,
        tokenizer=tokenizer,
        is_train=False,
    )

In [ ]:
# Define some useful utilities
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
from torchvision.ops.boxes import box_convert

COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

# Util to rescale predicted boxes to match image size
def rescale_boxes(boxes, size):
    img_w, img_h = size
    b = box_convert(boxes, "cxcywh", "xyxy")
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

# Plotting util function
def make_plot(pil_img, scores, boxes, labels):
    plt.figure(figsize=(16,10))
    np_image = np.array(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    assert len(scores) == len(boxes) == len(labels)
    for s, box, l, c in zip(scores, boxes, labels, colors):
        (xmin, ymin, xmax, ymax) = box.detach().numpy()
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        text = f'{l}: {s:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8))

    plt.imshow(np_image)
    plt.axis('off')
    plt.show()

# Postprocessing to rescale boxes and extract predicted spans of tokens
def postprocess_for_plot(outputs, text, tokenized, img_size, do_vqa=False):        
    model_outputs = outputs.model_output
    probs = 1 - model_outputs.pred_logits.softmax(-1)[0, :, -1]
    keep = (probs > 0.7)

    # convert boxes from [0; 1] to image scales
    boxes_scaled = rescale_boxes(model_outputs.pred_boxes[0, keep], img_size)

    # Extract the text spans predicted by each box
    positive_tokens = (model_outputs.pred_logits[0, keep].softmax(-1) > 0.1).nonzero().tolist()
    predicted_spans = defaultdict(str)
    for tok in positive_tokens:
        item, pos = tok
        if pos < 255:
            span = tokenized.token_to_chars(0, pos)
            predicted_spans[item] += " " + text[span.start:span.end]
    boxes_scaled = [boxes_scaled[int(k)] for k in sorted(list(predicted_spans.keys()))]
    labels = [predicted_spans[k] for k in sorted(list(predicted_spans.keys()))]
    
    return probs[keep], boxes_scaled, labels

# Main method to plot results of MDETR inference
def plot_inference_results(img, text, model=mdetr_pg, img_transform=img_transform, tokenizer=tokenizer, do_vqa=False):
    img_size = img.size
    # caption = text["caption"]
    tokenized = tokenizer.batch_encode_plus([text], padding="longest", return_tensors="pt")
    img_transformed = img_transform(img)
    outputs = model([img_transformed], tokenized["input_ids"])
    probs, boxes, labels = postprocess_for_plot(outputs, text, tokenized, img_size)
    make_plot(img, probs, boxes, labels)

In [ ]:
img, target = dataset[0]
text = target["caption"]

In [ ]:
img

In [ ]:
text

In [ ]:
plot_inference_results(img, text)

Note that the single phrase "a couple" yields several bounding boxes. More discussion on this point can be found in Appendix D of the [MDETR paper](https://arxiv.org/pdf/2104.12763.pdf). 

We can also pass custom prompts to see how well the model can detect other entities in the image.

In [ ]:
plot_inference_results(img, "Two lanterns")

In [ ]:
plot_inference_results(img, "A backpack and a blue railing")

In [ ]:
plot_inference_results(img, "A singer with a microphone")

In [ ]:
img, target = dataset[1]
text = target["caption"]

In [ ]:
img

In [ ]:
text

In [ ]:
plot_inference_results(img, text)

In [ ]:
plot_inference_results(img, "a white hat, trash bins, a pink headband, and blinds")

Note that the model will often output predictions even for objects not found in the image. As explained in the [demo](https://colab.research.google.com/github/ashkamath/mdetr/blob/colab/notebooks/MDETR_demo.ipynb) based on the original paper's repo, this is due to the lack of negatives in the training set.

In [ ]:
plot_inference_results(img, "A singer with a microphone")

## Visual Question Answering

During pretraining, MDETR uses a fixed number of object queries to detect different objects in the image. (The number of object queries provides an upper bound on the number of objects MDETR can detect.) For visual question answering, the object queries are supplemented by additional queries corresponding to each type of question (as well as one final query used for the question type itself). The resulting embeddings are fed to classification heads which perform predictions.

Here we demonstrate inference for a couple of samples from the [GQA dataset](https://cs.stanford.edu/people/dorarad/gqa/index.html). For GQA, the question types are based on GQA's set of [semantic types](https://cs.stanford.edu/people/dorarad/gqa/vis.html).

In [ ]:
# TorchMultimodal also provides a builder for VQA tasks.
from torchmultimodal.models.mdetr.model import mdetr_for_vqa

# The default values correspond to the GQA dataset.
mdetr_vqa = mdetr_for_vqa()

# But to perform VQA on another dataset, one can simply pass a different set of heads, e.g.
other_vqa_heads = nn.ModuleDict({"head1": nn.Linear(256, 3), "head2": nn.Linear(256,12)})
mdetr_other_vqa_dataset = mdetr_for_vqa(vqa_heads=other_vqa_heads)


checkpoint = torch.hub.load_state_dict_from_url(
            "https://pytorch.s3.amazonaws.com/models/multimodal/mdetr/gqa_resnet101_checkpoint.pth", map_location="cpu", check_hash=True
        )
mdetr_vqa.load_state_dict(checkpoint["model_ema"], strict=False)
mdetr_vqa.eval()
print("done")

In [ ]:
# Download a few sample images and corresponding annotations
import os
asset_dir = "assets/vqa"
img_dir = os.path.join(asset_dir, "images")
os.makedirs(img_dir, exist_ok=True)

!wget -nc "https://pytorch.s3.amazonaws.com/torchmultimodal/examples/mdetr/assets/gqa/images/n216553.jpg" -P $img_dir
!wget -nc "https://pytorch.s3.amazonaws.com/torchmultimodal/examples/mdetr/assets/gqa/images/n411121.jpg" -P $img_dir
!wget -nc "https://pytorch.s3.amazonaws.com/torchmultimodal/examples/mdetr/assets/gqa/gqa_sample_annotations.json" -P $asset_dir
!wget -nc "https://pytorch.s3.amazonaws.com/torchmultimodal/examples/mdetr/assets/gqa/gqa_answer2id.json" -P $asset_dir
!wget -nc "https://pytorch.s3.amazonaws.com/torchmultimodal/examples/mdetr/assets/gqa/gqa_answer2id_by_type.json" -P $asset_dir

In [ ]:
# Construct GQA dataset. Similar to for phrase grounding, we will perform
# the transforms ourselves for more flexible inference. 
from examples.mdetr.data.dataset import GQADataset
from pathlib import Path

dataset = GQADataset(
            img_dir,
            os.path.join(asset_dir, "gqa_sample_annotations.json"),
            transforms=None,
            return_tokens=True,
            tokenizer=tokenizer,
            ann_folder=Path(asset_dir),
        )

In [ ]:
# Define some more utils for plotting VQA predictions

# Used for displaying text answers
id2answerbytype = {}
for ans_type in dataset.answer2id_by_type.keys():                        
    curr_reversed_dict = {v: k for k, v in dataset.answer2id_by_type[ans_type].items()}
    id2answerbytype[ans_type] = curr_reversed_dict 
id2answer = {v: k for k, v in dataset.answer2id.items()}
types = {v: k for k, v in dataset.type_to_id.items()}
    
# Get predicted question type and answer along with ground truth
def get_question_answer(outputs, answer, types=types, id2answer=id2answer,id2answerbytype=id2answerbytype, top_k=5):
    preds = outputs.vqa_preds['answer_type'].softmax(-1)
    type_conf, type_pred = preds.max(-1)
    for i in range(preds.size(-1)):
        print(f"Question type {types[i]} probability: {round(100.0 * preds[:,i].item(), 2)}")
    type_pred = type_pred.item()
    answers = outputs.vqa_preds[f"answer_{types[type_pred]}"][0].softmax(-1)
    ans_conf, ans = torch.topk(answers, top_k)
    predicted_answers = [id2answerbytype[f"answer_{types[type_pred]}"][x.item()] for x in ans]
    ground_truth = id2answer[answer.item()]
    # ground_truth = id2answerbytype[f"answer_{types[answer_type.item()]}"][answer.item()]
    confidences =  [100 * type_conf.item() * x.item() for x in ans_conf]
    return predicted_answers, confidences, ground_truth
    
# Slight modification to original inference plotting function to also print QA predictions
def plot_qa_inference_results(img, question, answer, model=mdetr_vqa, img_transform=img_transform, tokenizer=tokenizer, top_k=5):
    img_size = img.size
    # caption = text["caption"]
    tokenized = tokenizer.batch_encode_plus([question], padding="longest", return_tensors="pt")
    img_transformed = img_transform(img)
    outputs = model([img_transformed], tokenized["input_ids"])
    probs, boxes, labels = postprocess_for_plot(outputs, question, tokenized, img_size)
    print(question)
    make_plot(img, probs, boxes, labels)
    predicted_answers, confidences, ground_truth = get_question_answer(outputs, answer, top_k=top_k)
    for i in range(len(predicted_answers)):
        print(f"Predicted answer #{i+1}: {predicted_answers[i]}, confidence: {round(confidences[i], 2)}")
    # print(f")
    # print(f"Predicted answer: {predicted_answers}\t confidence={[round(confidences, 2)}%")
    print(f"Ground truth answer: {ground_truth}")

In [ ]:
i = 0
img = dataset[i][0]
question = dataset[i][1]["caption"]
answer = dataset[i][1]["answer"]
plot_qa_inference_results(img, question, answer)

This sample has relation semantic type. We can see from the distribution of top predictions that the relation QA head is effective in anchoring to the text embeddings for left and right in the prompt.

We can also construct our own questions (assuming the answer is present in the set of labels):

In [ ]:
plot_qa_inference_results(img, "What animal is inside the backpack?", torch.tensor(dataset.answer2id["dog"]))

In [ ]:
i = 1
img = dataset[i][0]
question = dataset[i][1]["caption"]
answer = dataset[i][1]["answer"]
plot_qa_inference_results(img, question, answer)

Note that in this case the detector associates the answer with the question's interrogative word. This phenomenon is also discussed in Fig. 5 of the [MDETR paper](https://arxiv.org/pdf/2104.12763.pdf).

In [ ]:
plot_qa_inference_results(img, "What color is the man's bag?", torch.tensor(dataset.answer2id["blue"]))

## Conclusion

In this notebook, we introduced MDETR and showed how to use the implementation from TorchMultimodal. We demonstrated how to perform inference on a couple samples for phrase grounding and VQA tasks. For instructions on how to perform fine-tuning for VQA using TorchMultimodal, you can also check out the README here (add link).